In [1]:
# FILE HEADER SECTION ####################################
# 
# ui_module.py
# Implementation of UI for recommendation system
# Author:           ASHISH GARG
# Created on:       22-Nov-2019 18:17:53     
######################################################################################

# ------------------------------------- IMPORT EXTERNAL LIBRARY SECTION ---------------------
import similarity_module as simmod       
import load_dataset_module as loadmod     
import utility_module as utils    
from sys import stdout    
import math                     
# -------------------------------------------------------------------------------------------

# ----------------------------------- DECLARE CONSTANTS -------------------------------------
MOVINPUT1MESSAGE = "Enter 1st movie id [or] press SPACE bar and Enter for exit:"            
MOVINPUT2MESSAGE = "Enter 2nd movie id [or] press SPACE bar and Enter for exit:"            
USRINPUT1MESSAGE = "Enter 1st user id [or] press SPACE bar and Enter for exit:"               
USRINPUT2MESSAGE = "Enter 2nd user id [or] press SPACE bar and Enter for exit:"              
SELECTFUNCMESSAGE = "Choose similarity function for two users from the options provided:"   
MOVIDMESSAGE = "Enter movie id of which you want to see the details:"    
# --------------------------------------------------------------------------------------------

def call_usrsimilarity_function(usrpref, usrid1, usrid2, simfunc_code):
    try:
        similar_index = 0
        if(simfunc_code == utils.sim_func_code.EUCLIDEAN):    # call appropriate similarity function from the similarity module
            similar_index = simmod.euclidean_similarity(usrpref, usrid1, usrid2)
        elif(simfunc_code == utils.sim_func_code.COSINE):
            similar_index = simmod.cosine_similarity(usrpref, usrid1, usrid2)
        elif(simfunc_code == utils.sim_func_code.PEARSON):
            similar_index = simmod.pearson_similarity(usrpref, usrid1, usrid2)
        elif(simfunc_code == utils.sim_func_code.JACCARD):
            similar_index = simmod.jaccard_similarity(usrpref, usrid1, usrid2)
        elif(simfunc_code == utils.sim_func_code.MANHATTAN):
            similar_index = simmod.manhattan_similarity(usrpref, usrid1, usrid2)
        else: 
            similar_index = simmod.pearson_similarity(usrpref, usrid1, usrid2)
    except:
        print("Similarity function call between users exited with some error. Please check the entered value well!")
    finally:    
        return similar_index

# ------------------------------------------------- CORE FUNCTIONS SECTION ----------------------------------------------------
def check_if_rater_exist(raterid1,raterid2, usrpref):
    temp = (raterid1 in usrpref.keys() and raterid2 in usrpref.keys())
    return temp

def find_user_similarity(usrpref, selectsimfunc):
    usrid1 = utils.fetch_input(USRINPUT1MESSAGE)                                        # get user id 1 
    usrid2 = utils.fetch_input(USRINPUT2MESSAGE)                                        # get user id 2 
    validrater = check_if_rater_exist(usrid1, usrid2, usrpref)
    if validrater:
        if selectsimfunc == utils.similiarity_function_selection_mode.ENDUSER:
            utils.function_banner()                                                             # populate function code with message to user    
            simfunc_code = utils.fetch_input(SELECTFUNCMESSAGE)                                 # get selected similarity function code
            simfunc_code = int(simfunc_code)                                                    # convert similarity function code to integer
        elif selectsimfunc == utils.similiarity_function_selection_mode.ALL:
            man_index = call_usrsimilarity_function(usrpref, usrid1, usrid2, utils.sim_func_code.MANHATTAN)
            euc_index = call_usrsimilarity_function(usrpref, usrid1, usrid2, utils.sim_func_code.EUCLIDEAN)
            jac_index = call_usrsimilarity_function(usrpref, usrid1, usrid2, utils.sim_func_code.JACCARD)
            cos_index = call_usrsimilarity_function(usrpref, usrid1, usrid2, utils.sim_func_code.COSINE)
            pearson_index = call_usrsimilarity_function(usrpref, usrid1, usrid2, utils.sim_func_code.PEARSON)

            utils.clear_console()
            print(utils.colors.BOLD+"------------------------ USER SIMILARITY INDEX --------------------------------------"+utils.colors.END)
            print("Similarity between user id "+utils.colors.BOLD+str(usrid1)+utils.colors.END +" and "+utils.colors.BOLD+str(usrid2)+utils.colors.END+" are shown below:")
            print("Manhattan User Similarity Index: "+utils.colors.BOLD+str(man_index)+utils.colors.END)
            print("Euclidian User Similarity Index: "+utils.colors.BOLD+str(euc_index)+utils.colors.END)
            print("Jaccard User Similarity Index: "+utils.colors.BOLD+str(jac_index)+utils.colors.END)
            print("Cosine User Similarity Index: "+utils.colors.BOLD+str(cos_index)+utils.colors.END)
            print("Pearson User Similarity Index: "+utils.colors.BOLD+str(pearson_index)+utils.colors.END)
            print("-----------------------------------------------------------------------------------------------")
            utils.welcome_banner()
        else:
            simfunc_code = utils.sim_func_code.PEARSON

        if(selectsimfunc != utils.similiarity_function_selection_mode.ALL):
            simfunc_name = get_similarity_function_name(simfunc_code)                           # get function name selected by user

            # call similarity functions for users 
            similar_index = call_usrsimilarity_function(usrpref, usrid1, usrid2, simfunc_code)  

            # DISPLAY INFORMATION ON SCREEN
            utils.clear_console()   

            if(str(similar_index) == str(math.nan)):
                print(simfunc_name.upper(),'similarity function'.upper(), 'can''t handle this vector appropriately')
            else:
                print(utils.colors.BOLD+simfunc_name+" similarity index between rater "+str(usrid1) +" and rater " +str(usrid2) +" is "+str(similar_index)+utils.colors.END)
            utils.welcome_banner()
    else:
        utils.clear_console()
        utils.welcome_banner()
        print(utils.colors.RED+"Provided user id is not valid!"+utils.colors.END)
        
def check_if_movie_exist(movid1,movid2, movwatchers):
    temp = (movid1 in movwatchers.keys() and movid2 in movwatchers.keys())
    return temp
    
def find_movie_similarity(usrpref, movwatchers,genre_details, selectsimfunc):
    movid1 = utils.fetch_input(MOVINPUT1MESSAGE)        # take movie id 1 from end user 
    movid2 = utils.fetch_input(MOVINPUT2MESSAGE)        # take movie id 2 from end user 
    validmovieid = check_if_movie_exist(movid1, movid2, movwatchers)
    if(validmovieid):
        movtitle1 = movwatchers[movid1]['title']
        movtitle2 = movwatchers[movid2]['title']
        movie1_genres =  loadmod.fetch_genresformovie(movwatchers, genre_details, movid1)
        movie2_genres =  loadmod.fetch_genresformovie(movwatchers, genre_details, movid2)

        if selectsimfunc == utils.similiarity_function_selection_mode.ENDUSER:
            utils.function_banner()                             # populate function list to user 
            simfunc_code = utils.fetch_input(SELECTFUNCMESSAGE) # call utility function to get function code from user 
            simfunc_code = int(simfunc_code)                    # convert simfunc_code to integer    
        elif selectsimfunc == utils.similiarity_function_selection_mode.ALL:
            man_index = simmod.find_movie_similarity(usrpref, movwatchers,movtitle1, movtitle2,utils.sim_func_code.MANHATTAN)
            euc_index = simmod.find_movie_similarity(usrpref, movwatchers,movtitle1, movtitle2,utils.sim_func_code.EUCLIDEAN)
            jac_index = simmod.find_movie_similarity(usrpref, movwatchers,movtitle1, movtitle2,utils.sim_func_code.JACCARD)
            cos_index = simmod.find_movie_similarity(usrpref, movwatchers,movtitle1, movtitle2,utils.sim_func_code.COSINE)
            pearson_index = simmod.find_movie_similarity(usrpref, movwatchers,movtitle1, movtitle2,utils.sim_func_code.PEARSON)

            utils.clear_console()
            print(utils.colors.BOLD+"------------------------ MOVIE SIMILARITY INDEX --------------------------------------"+utils.colors.END)
            print("Similarity between movie id "+utils.colors.BOLD+str(movid1)+utils.colors.END +" and "+utils.colors.BOLD+str(movid2)+utils.colors.END+" are shown below:")
            print("Manhattan Movie Similarity Index: "+utils.colors.BOLD+str(man_index)+utils.colors.END)
            print("Euclidian Movie Similarity Index: "+utils.colors.BOLD+str(euc_index)+utils.colors.END)
            print("Jaccard Movie Similarity Index: "+utils.colors.BOLD+str(jac_index)+utils.colors.END)
            print("Cosine Movie Similarity Index: "+utils.colors.BOLD+str(cos_index)+utils.colors.END)
            print("Pearson Movie Similarity Index: "+utils.colors.BOLD+str(pearson_index)+utils.colors.END)
            print("-----------------------------------------------------------------------------------------------")
            utils.welcome_banner()
        else: 
            simfunc_code = utils.sim_func_code.COSINE


        if (selectsimfunc != utils.similiarity_function_selection_mode.ALL):
            simfunc_name = get_similarity_function_name(simfunc_code)
            corr_coeff = simmod.find_movie_similarity(usrpref, movwatchers,movtitle1, movtitle2,simfunc_code)  # call movie similarity function from similarity_module

            # DISPLAY DATA ON SCREEN
            utils.clear_console()
            print("-----------------------------------------------------------------------------------------------")
            if(str(corr_coeff) == str(math.nan)):
                print(simfunc_name.upper(),'similarity function'.upper(), 'can''t handle this vector appropriately!')
            else:
                print(utils.colors.BOLD+simfunc_name+" similarity index between movie id "+str(movid1)+" and movie id "+str(movid2)+" is "+str(corr_coeff)+utils.colors.END)   
            print("-----------------------------------------------------------------------------------------------")
            print("Movie 1 details:")
            print("Name: " +movtitle1)
            print('Genres: ', [str(name) for name in movie1_genres])
            print('-----------------------------------------------------------------------------------------------')
            print("Movie 2 details:")
            print("Name: " +movtitle2)
            print('Genres: ', [str(name) for name in movie2_genres])
            print("-----------------------------------------------------------------------------------------------")                        
            utils.welcome_banner()
    else:
        utils.clear_console()
        utils.welcome_banner()
        print(utils.colors.RED+"Provided movie id is not valid!"+utils.colors.END)
        
def get_similarity_function_name(funccode):
    if(funccode == 1):
        return 'EUCLIDEAN'
    elif(funccode == 2):
        return 'COSINE'
    elif(funccode == 3):
        return 'PEARSON'
    elif(funccode == 4):
        return 'JACCARD'
    elif(funccode == 5):
        return 'MANHATTAN'
    else:
        return ""

def print_movie_details(movdataset, genredataset, movid):   
    utils.clear_console()
    validmovieid = movid in movdataset
    if(validmovieid):
        title = movdataset[movid]['title']  # retrieve movie title
        genres = loadmod.fetch_genresformovie(movdataset, genredataset, movid) # retrieve movie genres names
        totalwatchers = len(movdataset[movid] ['userids']) # retrieve total number of watchers
        print(utils.colors.GREEN+"------------------------------------ MOVIE DETAILS ---------------------------------------------")
        print("MOVIE ID:", movid)
        print("MOVIE NAME:", title)
        print("MOVIE GENRES:", [str(name) for name in genres])
        print("TOTAL WATCHERS:", str(totalwatchers))
        print("-----------------------------------------------------------------------------------------------"+utils.colors.END)
    else:
        print(utils.colors.RED+"Provided movie id is not valid!"+utils.colors.END)
    utils.welcome_banner()
    
# ------------------------------------------------ MAIN FUNCTION SECTION ---------------------------------------------------
def main():
    try: 
        utils.welcome_banner()                                                    # populate welcome header

        # create datasets that would be used across the applications
        user_preference = loadmod.load_usersdetails()                  # load user preference data to the dictionary    
        movies_genres_and_watchers = loadmod.load_moviedetails()    # load movie data to the dictionary    
        genre_details = loadmod.load_genredetails()                 # load genre postion with corresponding name to the dictionary     

        # till user won't enter the valid value or opt for EXIT user stay here
        wanna_exit = False
        while not wanna_exit:
            usrinput = input('Select operation from above: ') # Prompt user to select the operations in code 
            if(not usrinput.isspace()):                       # check if user input is not SPCE+ENTER, else exit 
                valid_input = True if utils.valid_number(usrinput) else False # check if input is valid or not
                if(valid_input):
                    usrinput = int(usrinput)
                    if (usrinput == 1):  
                        find_user_similarity(user_preference, utils.similiarity_function_selection_mode.ENDUSER) # do user similarity check and set                             
                    elif(usrinput == 2):            
                        find_movie_similarity(user_preference, movies_genres_and_watchers, genre_details, utils.similiarity_function_selection_mode.ENDUSER) # do movie similarity check
                    elif(usrinput == 3):
                        movid = utils.fetch_input(MOVIDMESSAGE)   # retrieve movie id from user
                        print_movie_details(movies_genres_and_watchers, genre_details, movid)
                    elif(usrinput == 4):                        
                        find_user_similarity(user_preference, utils.similiarity_function_selection_mode.BEST) # Call best user similarity function  
                    elif(usrinput == 5):            
                        find_movie_similarity(user_preference, movies_genres_and_watchers, genre_details, utils.similiarity_function_selection_mode.BEST) # do movie similarity check
                    elif(usrinput == 6):            
                        find_user_similarity(user_preference, utils.similiarity_function_selection_mode.ALL) # do user similarity check and set                              
                    elif(usrinput == 7):            
                        find_movie_similarity(user_preference, movies_genres_and_watchers, genre_details, utils.similiarity_function_selection_mode.ALL) # do movie similarity check
                    else:
                        print('Enter correct key!')
                else:
                    print("Valid inputs are - 1,2,3,4,5,6 or 7") 
            else:
                wanna_exit = True                     
    except Exception as e: 
        print("Exiting Program!") 

main()

 ***** WELCOME TO MOVIE RECOMMENDATION SYSTEM (SELECT OPERATIONS AS CODE) ****
1. Perform USER SIMILARITY
2. Perform MOVIE SIMILARITY
3. Get MOVIE DETAILS
4. Perform USER SIMILARITY (best fit)
5. Perform MOVIE SIMILARITY (best fit)
6. Perform USER SIMILARITY (All functions)
7. Perform MOVIE SIMILARITY (All functions)
--> Press SPACE bar with enter key to EXIT
***********************************************************************
Provided movie id is not valid!


Select operation from above:   
